In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import numpy as np

# Pandas私房手册-Pandas必知必会

## 属性和底层数据

`pandas`对象(`Index`、`Series`、`DataFrame`)都可以看作数组的容器，数组保存实际数据并执行实际计算。对于许多类型，底层数组是`numpy.ndarray`。但是，`pandas`和第三方库扩展了`NumPy`的类型系统来添加对定制数组的支持。  

### `series`和`index` 获取底层数据

`pandas`有一些属性方法可以让你接触到它的底层数据，比如要访问`index`，`Series`等对象的实际数据，可以使用`.array`特性：

In [17]:
index = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])

s.array
s.index.array

<PandasArray>
[-1.5674189575123696, -1.0830671011867505, -0.4227460650968681,
  1.0529532404941764,  0.9104896300810849]
Length: 5, dtype: float64

<PandasArray>
['a', 'b', 'c', 'd', 'e']
Length: 5, dtype: object

可以看到，返回的是`PandasArray`对象，它对`numpy`数组对象进行浅层包装，如果想要获得原生的`numpy`数组，可以使用`.to_numpy()`或者`numpy.asarray()`方法：

In [8]:
s.to_numpy()
np.asarray(s)

array([ 0.52894286,  0.01071537, -0.56884353, -0.77302582,  0.64477244])

array([ 0.52894286,  0.01071537, -0.56884353, -0.77302582,  0.64477244])

因为`pandas`包含比`numpy`更丰富的数据类型`dtype`，因此，使用`to_numpy()`时，可能会对结果的`dtype`进行一些控制。比如，带时区的`datetimes`。`NumPy`没有表示包含时区信息的`datetimes`类型，此时可能有两种表示：
- 一个`object-dtype`数据类型，包含时间戳对象的数组，每个对象具有正确的时区`tz`属性。
- 一个包含`datetime64[ns]`对象的数组，所有的值值已转换为UTC标准时间，并丢弃了时区信息。

In [9]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser.to_numpy(dtype=object)
ser.to_numpy(dtype="datetime64[ns]")

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

### `dataframe`获取底层数据

从`DataFrame`中获取原始数据可能要复杂一些。如果`DataFrame`的所有列只有一个数据类型时，`DataFrame.to_numpy()`直接返回底层数据，而且此时返回的是一个视图，即对返回的数组执行的是就地修改，这些更改将反映在原始的`dataframe`中：

In [18]:
# 原始的dataframe
df
# 返回一个数组
nda = df.to_numpy()
# 修改数组
nda[0, 0] = 1
# 原始数组也被修改了
df

,A,B,C
2000-01-01,0.105807,-0.495605,1.085258
2000-01-02,-1.161728,0.985065,-0.095596
2000-01-03,-0.435239,1.236400,-1.262306
2000-01-04,1.009800,-0.071548,-1.091554
2000-01-05,1.181627,0.006865,-0.764698
2000-01-06,-1.943281,1.134871,1.014496
2000-01-07,-1.079137,-1.548577,0.571199
2000-01-08,0.058927,-0.556218,-0.711983


,A,B,C
2000-01-01,1.000000,-0.495605,1.085258
2000-01-02,-1.161728,0.985065,-0.095596
2000-01-03,-0.435239,1.236400,-1.262306
2000-01-04,1.009800,-0.071548,-1.091554
2000-01-05,1.181627,0.006865,-0.764698
2000-01-06,-1.943281,1.134871,1.014496
2000-01-07,-1.079137,-1.548577,0.571199
2000-01-08,0.058927,-0.556218,-0.711983


对于异构数据（例如，一些`DataFrame`的列并不都是相同的`dtype`数据类型），情况就不一样了。`to_numpy`返回的是一个副本而不是一个视图，修改返回的结果对原数据没有影响：

In [31]:
df = DataFrame({'A':['a', 'b', 'c'], 'B':[1, 2, 3]})
df
nda = df.to_numpy()
nda[0, 0] = 1
df
nda

,A,B
0,a,1
1,b,2
2,c,3


,A,B
0,a,1
1,b,2
2,c,3


array([[1, 1],
       ['b', 2],
       ['c', 3]], dtype=object)

以前，`pandas`推荐使用`.values()`方法获取`numpy`数组，即便是现在，仍然保留了这个方法，而且官网上面也有相关的文档说明，但是现在推荐使用`.array`或者`.to_numpy()`特性或者方法。因为`.values()`方法存在以下缺点：
1. 当`series`的`dtype`是扩展类型（比如前面提到过的`Timestamp`类型）时，`.values()`方法将始终返回一个`ExtensionArray`，并且永远不会复制数据。而`to_numpy()`是可以对返回的类型进行控制的。
2. 当`dataframe`包含多种数据类型时，`DataFrame.value()`可能会复制数据并强制转换数据的`dtype`类型，这操作比较消耗性能。`to_numpy()`可以清楚地表明返回的是`NumPy`数组，可能不是`DataFrame`中相同数据的视图，简单来说就是意思比较清楚，不容易有歧义。

In [39]:
ser
ser.values
ser.to_numpy()
ser.to_numpy('datetime64[ns]')

0   2000-01-01 00:00:00+01:00
1   2000-01-02 00:00:00+01:00
dtype: datetime64[ns, CET]

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

## 加速操作

`pandas`支持使用`numexpr`库和`bottleneck`库加速某些类型的二进制数字和布尔运算。这些库在处理大型数据集时特别有用，并提供了较大的速度提升。`numexpr`使用智能分块、缓存和多核。`bottleneck`是一组特殊的`cython`例程(`routines`)，在处理具有`nan`的数组时，这些例程特别快，`pandas`默认使用这些库，但需要先安装依赖的第三方库，`0.20`版本以后，可以通过设置选项来进行控制：

In [42]:
df1 = DataFrame(np.random.randn(100, 100000))
df2 = DataFrame(np.random.randn(100, 100000))

pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)

In [43]:
%timeit df1 > df2

24.2 s ± 753 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


当打开这些功能时，在我的机器上（i79750 12核，16G内存），速度提升了1000倍：

In [44]:
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [41]:
%timeit df1 > df2

26.9 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 灵活的二元操作

使用`dataframe`进行二元计算，有2点需要注意：
1. 高维对象和低维对象计算时的广播行为。
2. 计算时候对于缺失值的处理。

### 匹配/广播行为

相同形状的`DataFrame`对象进行计算，使用常用的数学符号的`+,-,*,/`直接进行计算就可以了，但是对于不同维度的对象进行计算，则需要使用`pandas`提供`add()`、`sub()`、`mul()`、`div()`以及相应的`radd()`、`rsub()`等方法，同时通过`axis`关键字匹配索引或列来实现:

In [10]:
df = pd.DataFrame({
    'one':
    pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
    'two':
    pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
    'three':
    pd.Series(np.random.randn(3), index=['b', 'c', 'd'])
})

row = df.iloc[1]
column = df['two']

df
# 注意此时减去row，为水平方向，axis要设置为columns或者1
df.sub(row, axis='columns')
# 减去列，为垂直方向，axis设置为index或者0
df.sub(column, axis='index')

,one,two,three
a,-0.411602,1.195882,NaN
b,0.157751,0.267277,1.792477
c,1.570567,-0.941445,1.004975
d,NaN,-0.410826,-0.213466


,one,two,three
a,-0.569353,0.928606,NaN
b,0.000000,0.000000,0.000000
c,1.412816,-1.208721,-0.787502
d,NaN,-0.678103,-2.005944


,one,two,three
a,-1.607484,0.0,NaN
b,-0.109526,0.0,1.525200
c,2.512011,0.0,1.946420
d,NaN,0.0,0.197359


`+,-,*,/`其实也可以和低维的对象进行计算，但是只会把低维对象当作行，如果是`series`对象，会严格按照标签进行匹配，所以可能会得到意料之外的结果：

In [19]:
df - row
# 把column看成一行，并且标签进行对齐，df的标签是one,two,three,而columns的标签是a,b,c,d，因此结果全部都是NaN
df - column

,one,two,three
a,-0.569353,0.928606,NaN
b,0.000000,0.000000,0.000000
c,1.412816,-1.208721,-0.787502
d,NaN,-0.678103,-2.005944


,a,b,c,d,one,three,two
a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN,NaN,NaN,NaN


`series`和`index`都支持`python`内置的`divmod()`函数，返回一个元组，分别是整除结果构成的序列和余数构成的序列：

In [21]:
s = pd.Series(np.arange(5))
div, rem = divmod(s, 3)
div
rem

0    0
1    0
2    0
3    1
4    1
dtype: int32

0    0
1    1
2    2
3    0
4    1
dtype: int32

它的第二个参数也可以是和`series`或者`index`等长的序列：

In [23]:
div, rem = divmod(s, [1, 1, 1, 2, 2])
div
rem

0    0
1    1
2    2
3    1
4    2
dtype: int32

0    0
1    0
2    0
3    1
4    0
dtype: int32

### 包含缺失值的操作

在`Series`和`DataFrame`中，所有二元运算的函数都有一个`fill_value`的参数，表示当某个位置上的值最多有一个丢失时，要替换的值。例如，当两个`dataframe`对象相加时，将`NaN`视为0，除非两个`dataframe`都缺少该值（此时结果将是`NaN`，不过可以稍后使用`fillna`将`NaN`替换为其他值）：

In [29]:
df2 = df.copy()
df2.loc['a', 'three'] = 1
df
df2
# 默认情况下两个dataframe相加，如果存在NaN，则结果也是NaN
df + df2

,one,two,three
a,-0.411602,1.195882,NaN
b,0.157751,0.267277,1.792477
c,1.570567,-0.941445,1.004975
d,NaN,-0.410826,-0.213466


,one,two,three
a,-0.411602,1.195882,1.000000
b,0.157751,0.267277,1.792477
c,1.570567,-0.941445,1.004975
d,NaN,-0.410826,-0.213466


,one,two,three
a,-0.823204,2.391765,NaN
b,0.315502,0.534554,3.584954
c,3.141133,-1.882889,2.009951
d,NaN,-0.821652,-0.426933


`fill_value`表示填充缺失的值，除非这个位置上的值全部都是`NaN`，此时结果也是`NaN`：

In [32]:
df.add(df2, fill_value=0)
df.add(df, fill_value=0)

,one,two,three
a,-0.823204,2.391765,1.000000
b,0.315502,0.534554,3.584954
c,3.141133,-1.882889,2.009951
d,NaN,-0.821652,-0.426933


,one,two,three
a,-0.823204,2.391765,NaN
b,0.315502,0.534554,3.584954
c,3.141133,-1.882889,2.009951
d,NaN,-0.821652,-0.426933
